In [1]:
import pandas as pd
import numpy as np
import datetime

# Load 
## Covid-19 data

In [2]:
OxCGRT = pd.read_csv("https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv")
OxCGRT.loc[:,'Date'] = pd.to_datetime(OxCGRT.Date, format='%Y%m%d')

C:\Users\37125\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Region data

In [3]:
regions = pd.read_csv("https://datahub.io/JohnSnowLabs/country-and-continent-codes-list/r/country-and-continent-codes-list-csv.csv")

In [4]:
regions.head()

,Continent_Name,Continent_Code,Country_Name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,"Afghanistan, Islamic Republic of",AF,AFG,4.0
1,Europe,EU,"Albania, Republic of",AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,"Algeria, People's Democratic Republic of",DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0


Merge continent names to the main data set (merge on country code)

In [5]:
regions = regions[["Three_Letter_Country_Code","Continent_Name"]]
OxCGRT = pd.merge(OxCGRT, regions, how='left', left_on='CountryCode', right_on='Three_Letter_Country_Code')

Remove extra columns and rows (countries with atypical data)

In [6]:
columns_of_interest = ['CountryName', 'CountryCode', 'Continent_Name', 'Date', 'ConfirmedCases', 'ConfirmedDeaths',
                       'C1_School closing','C2_Workplace closing','C3_Cancel public events',
                       'C4_Restrictions on gatherings', 'C5_Close public transport','C6_Stay at home requirements',
                       'C7_Restrictions on internal movement','C8_International travel controls',
                       'E1_Income support','E2_Debt/contract relief', 'E3_Fiscal measures','E4_International support', 
                       'H1_Public information campaigns','H2_Testing policy', 'H3_Contact tracing','H4_Emergency investment in healthcare', 
                       'H5_Investment in vaccines','H6_Facial Coverings', 'H7_Vaccination policy', 'H8_Protection of elderly people', 
                       'M1_Wildcard',
                       'StringencyIndex','StringencyIndexForDisplay','StringencyLegacyIndex','StringencyLegacyIndexForDisplay', 
                       'GovernmentResponseIndex','GovernmentResponseIndexForDisplay', 
                       'ContainmentHealthIndex','ContainmentHealthIndexForDisplay', 
                       'EconomicSupportIndex','EconomicSupportIndexForDisplay']
excluded_countries = ['United Kingdom','Canada','Brazil','United States', # because have observations by regions
                      'Kosovo', 'Taiwan'] # No population data

data = OxCGRT[~OxCGRT.CountryName.isin(excluded_countries)][columns_of_interest]

In [7]:
data.head(3)

,CountryName,CountryCode,Continent_Name,Date,ConfirmedCases,ConfirmedDeaths,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,...,StringencyIndex,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay
0,Aruba,ABW,North America,2020-01-01,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,North America,2020-01-02,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,North America,2020-01-03,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Tests data

In [8]:
tests = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv")

In [9]:
tests.loc[:,~tests.columns.isin(["Entity","Source URL","Source label","Notes"])].head(3)

,ISO code,Date,Daily change in cumulative total,Cumulative total,Cumulative total per thousand,Daily change in cumulative total per thousand,7-day smoothed daily change,7-day smoothed daily change per thousand,Short-term positive rate,Short-term tests per case
0,ALB,2020-02-25,8.0,8.0,0.003,0.003,NaN,NaN,NaN,NaN
1,ALB,2020-02-26,5.0,13.0,0.005,0.002,NaN,NaN,NaN,NaN
2,ALB,2020-02-27,4.0,17.0,0.006,0.001,NaN,NaN,NaN,NaN


In [10]:
tests.loc[:,'Date'] = pd.to_datetime(tests.Date, format='%Y-%m-%d')
tests = tests.loc[:,~tests.columns.isin(["Entity","Source URL","Source label","Notes"])]
data = pd.merge(data, tests, how='left', left_on=["CountryCode","Date"], right_on=["ISO code","Date"])

## Vaccination data

In [11]:
vaccine = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")

In [12]:
vaccine.head(2)

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0


In [13]:
vaccine.loc[:,'date'] = pd.to_datetime(vaccine.date, format='%Y-%m-%d')
vaccine = vaccine.loc[:,vaccine.columns != "location"]
data = pd.merge(data, vaccine, how='left', left_on=["CountryCode","Date"], right_on=["iso_code","date"])

## Population data

In [14]:
# Source: https://data.worldbank.org/indicator/SP.POP.TOTL
population = pd.read_csv("data/population.csv",skiprows=4)
population = population[["Country Code","2019"]]
# Get country names from 'data' table to ensure identical naming
population = pd.merge(population, data[['CountryCode','CountryName']].drop_duplicates(),
                      left_on='Country Code', right_on='CountryCode')
population = population[["CountryName","2019"]].set_index("CountryName")
population.index.name = None

In [15]:
population.head()

,2019
Aruba,106314.0
Afghanistan,38041754.0
Angola,31825295.0
Albania,2854191.0
Andorra,77142.0


# Calculate
## Daily cases

In [16]:
for country in data.CountryName.unique():
    data.loc[(data.CountryName==country),"DailyCases"] = data[(data.CountryName==country)]["ConfirmedCases"].diff()

## 7-day-average

In [17]:
for country in data.CountryName.unique():
    data.loc[(data.CountryName==country),"Average7"] = data[(data.CountryName==country)]["ConfirmedCases"].diff().rolling(7).mean()

## 14-day-average

In [18]:
for country in data.CountryName.unique():
    data.loc[(data.CountryName==country),"Average14"] = data[(data.CountryName==country)]["ConfirmedCases"].diff().rolling(14).mean()

## Daily cases per 100K population

In [19]:
for country in data.CountryName.unique():
    data.loc[(data.CountryName==country),"DailyCases_100K"] = data[(data.CountryName==country)]["DailyCases"] / population.loc[country].values * 100000

## 7-day-average per 100K population

In [20]:
for country in data.CountryName.unique():
    data.loc[(data.CountryName==country),"Average7_100K"] = data[(data.CountryName==country)]["Average7"] / population.loc[country].values * 100000

## 14-day-average per 100K population

In [21]:
for country in data.CountryName.unique():
    data.loc[(data.CountryName==country),"Average14_100K"] = data[(data.CountryName==country)]["Average14"] / population.loc[country].values * 100000

# Save to the file

In [22]:
data.to_csv("data/OxCGRT_latest.csv")